# 1. Get an wrds account

[Request an account](http://scrc.scrc.nyu.edu/scrc/?page_id=1395)

# 2. Connect to wrds via Python
Full documentation on wrds [website](https://wrds-www.wharton.upenn.edu/pages/support/programming-wrds/programming-python/python-from-your-computer/). [Github Source](https://github.com/wharton/wrds)

Also supports R, Matlab, SAS, PostgreSQL, see [here](https://wrds-www.wharton.upenn.edu/pages/support/programming-wrds/)

In [1]:
import wrds
# help(wrds.Connection)
db = wrds.Connection(wrds_username='hanyuzhang')
# The pgpass file saves the username and password and allows you to log in with just the username 
# when calling the connection, i.e. the password is only needed the first time we connect to wrds. 
# db.create_pgpass_file()

Enter your WRDS username [hzhang]:hanyuzhang
Enter your password:········
WRDS recommends setting up a .pgpass file.
You can find more info here:
https://www.postgresql.org/docs/9.5/static/libpq-pgpass.html.
Loading library list...
Done


Common Methods:
- db.connection()
- db.list_libraries()
- db.list_tables()
- db.describe_table()
- db.get_table()
- db.raw_sql()
- db.close()
- db.get_row_count()

To query the dataset structure (Metadata):

1. List all available libraries at WRDS using list_libraries()
2. Select a library, and list all available datasets within that library using list_tables()
3. Select a table, and view all available variables (column headers) within that dataset using describe_table()
4. Query data use get_table() or raw_sql()*
5. Data results are always returned as a **Pandas DataFrame**

**Note**: library and table names must be in lowercases. A comprensive list of wrds libraries is avavilable [here](https://wrds-www.wharton.upenn.edu/pages/about/data-vendors/) but not every one is necessarily available due to subscription limit.

*: A [tutorial](https://www.tutorialspoint.com/sql/sql-select-query.htm) on commonly used SQL syntax (SELECT, WHERE, AND, OR, IN etc.)

In [5]:
db.list_libraries()[:5]

['wrds_lib_internal',
 'ppubsamp_d4d_new',
 'ktmine',
 'ibescorp',
 'contrib_char_returns_new']

In [3]:
db.list_tables("crsp")[:5]

['acti', 'asia', 'asib', 'asic', 'asio']

In [6]:
db.describe_table("crsp", 'msf')

Approximately 4602260 rows in crsp.msf.


,name,nullable,type
0,cusip,True,VARCHAR
1,permno,True,DOUBLE PRECISION
2,permco,True,DOUBLE PRECISION
3,issuno,True,DOUBLE PRECISION
4,hexcd,True,DOUBLE PRECISION
5,hsiccd,True,DOUBLE PRECISION
6,date,True,DATE
7,bidlo,True,DOUBLE PRECISION
8,askhi,True,DOUBLE PRECISION
9,prc,True,DOUBLE PRECISION


In [5]:
db.raw_sql('SELECT * FROM crsp.msf LIMIT 5;', date_cols=['date'])

,cusip,permno,permco,issuno,hexcd,hsiccd,date,bidlo,askhi,prc,...,ret,bid,ask,shrout,cfacpr,cfacshr,altprc,spread,altprcdt,retx
0,68391610,10000.0,7952.0,10396.0,3.0,3990.0,1985-12-31,NaN,NaN,NaN,...,NaN,None,None,NaN,NaN,NaN,-2.5625,NaN,1986-01-07,NaN
1,68391610,10000.0,7952.0,10396.0,3.0,3990.0,1986-01-31,-2.50,-4.4375,-4.3750,...,NaN,None,None,3680.0,1.0,1.0,-4.3750,0.250,1986-01-31,NaN
2,68391610,10000.0,7952.0,10396.0,3.0,3990.0,1986-02-28,-3.25,-4.3750,-3.2500,...,-0.257143,None,None,3680.0,1.0,1.0,-3.2500,0.250,1986-02-28,-0.257143
3,68391610,10000.0,7952.0,10396.0,3.0,3990.0,1986-03-31,-3.25,-4.4375,-4.4375,...,0.365385,None,None,3680.0,1.0,1.0,-4.4375,0.125,1986-03-31,0.365385
4,68391610,10000.0,7952.0,10396.0,3.0,3990.0,1986-04-30,-4.00,-4.3125,-4.0000,...,-0.098592,None,None,3793.0,1.0,1.0,-4.0000,0.250,1986-04-30,-0.098592


# 3. Get price data
- crsp.dsf
- crsp.msf
- ff.factors_daily (useful as benchmark)
- etc.

An overview of CRSP: [link](https://wrds-www.wharton.upenn.edu/pages/support/manuals-and-overviews/crsp/stocks-and-indices/overview-crsp-us-stock-database/)

TL;DR:
1. In CRSP, companies are uniquely defined by `permco` and shares by `permno`. A `permco` may correspond to more than one `permno`. [More](https://libguides.stanford.edu/c.php?g=559845&p=6686228)
2. **negative stock price (prc)**: If the closing price is not available for any given period, the number in the price field is replaced with a bid/ask average. Bid/ask averages have dashes placed in front of them. These do not incorrectly reflect negative prices; they serve to distinguish bid/ask averages from actual closing prices. If neither the price nor bid/ask average is available, the field is set to zero.
3. **Adjusting for stock Splits and other corporate actions**: returns are already adjusted for splits, but prices and shares outstanding are not. To adjust, do the following (CFACSHR is not always equal to CFACPR. This can be caused by less common distribution events, spin-offs, and rights):
    - Adjusted prices = PRC / CFACPR
    - Adjusted shares = SHROUT * CFACSHR
4. **share code (shrcd)**: a SHRCD of 10 or 11 would represent U.S. common stocks.

**Note**: NYU's subscription only has access to the annually updated CRSP database which is updated every Feburary, i.e. price history is only available through end of 2019 (as of Nov. 2020). See end of section for a web-scraping script to download price history beyond 2019 from yahoo finance (for currently listed companies).

In [6]:
# an example: get end-of-month price of certain stocks
# first we need a list of stocks to get price data for, identified by permno
# one way is to use crsp's stock header file, which contains all previously and currently listed stocks
# and narrow it down from there
db.describe_table('crsp', 'dsfhdr')

Approximately 33584 rows in crsp.dsfhdr.


,name,nullable,type
0,permno,True,DOUBLE PRECISION
1,permco,True,DOUBLE PRECISION
2,hshrcd,True,DOUBLE PRECISION
3,dlstcd,True,DOUBLE PRECISION
4,hcusip,True,VARCHAR
5,htick,True,VARCHAR
6,hcomnam,True,VARCHAR
7,htsymbol,True,VARCHAR
8,hnaics,True,VARCHAR
9,hprimexc,True,VARCHAR


In [12]:
# select U.S common stocks, with price history available after start of 2015
stock_header = db.raw_sql("SELECT permno FROM crsp.dsfhdr WHERE hshrcd IN (10,11) AND begdat > '2015-01-01'")
permno_list = list(set(stock_header['permno'].tolist()))
db.raw_sql("select * from crsp.dsfhdr where htsymbol='SPY' limit 1")

,permno,permco,hshrcd,dlstcd,hcusip,htick,hcomnam,htsymbol,hnaics,hprimexc,...,begvol,endvol,begbid,endbid,begask,endask,begopr,endopr,hsicmg,hsicig
0,84398.0,46699.0,73.0,100.0,78462F10,SPY,SPDR S & P 500 E T F TRUST,SPY,525990,R,...,1993-01-29,2019-12-31,1993-02-01,2019-12-31,1993-02-01,2019-12-31,1993-01-29,2019-12-31,67.0,672.0


In [8]:
db.describe_table('crsp', 'msf')

Approximately 4602260 rows in crsp.msf.


,name,nullable,type
0,cusip,True,VARCHAR
1,permno,True,DOUBLE PRECISION
2,permco,True,DOUBLE PRECISION
3,issuno,True,DOUBLE PRECISION
4,hexcd,True,DOUBLE PRECISION
5,hsiccd,True,DOUBLE PRECISION
6,date,True,DATE
7,bidlo,True,DOUBLE PRECISION
8,askhi,True,DOUBLE PRECISION
9,prc,True,DOUBLE PRECISION


In [9]:
# get historical monthly returns for all stocks in permno_list
# if returned dataframe is not huge, you can save it to local for faster reading
db.raw_sql(f"SELECT * FROM crsp.msf "
           f"where permno in ({','.join(str(p) for p in permno_list)})"
           f"AND prc IS NOT NULL")

,cusip,permno,permco,issuno,hexcd,hsiccd,date,bidlo,askhi,prc,...,ret,bid,ask,shrout,cfacpr,cfacshr,altprc,spread,altprcdt,retx
0,70338W10,15144.0,55157.0,75508.0,3.0,9999.0,2015-02-27,21.889999,26.340000,22.530001,...,NaN,22.490000,22.780001,3234.0,1.0,1.0,22.530001,NaN,2015-02-27,NaN
1,70338W10,15144.0,55157.0,75508.0,3.0,9999.0,2015-03-31,21.990000,24.969999,24.969999,...,0.108300,24.340000,24.820000,3266.0,1.0,1.0,24.969999,NaN,2015-03-31,0.108300
2,70338W10,15144.0,55157.0,75508.0,3.0,9999.0,2015-04-30,24.620001,26.260000,25.809999,...,0.033640,25.809999,25.900000,3266.0,1.0,1.0,25.809999,NaN,2015-04-30,0.033640
3,70338W10,15144.0,55157.0,75508.0,3.0,9999.0,2015-05-29,25.010000,25.930000,25.790001,...,-0.000775,25.750000,25.950001,3266.0,1.0,1.0,25.790001,NaN,2015-05-29,-0.000775
4,70338W10,15144.0,55157.0,75508.0,3.0,9999.0,2015-06-30,24.100000,26.250000,24.650000,...,-0.044203,24.500000,25.280001,3266.0,1.0,1.0,24.650000,NaN,2015-06-30,-0.044203
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28020,53952P10,19139.0,56975.0,79627.0,3.0,9999.0,2019-12-31,5.360000,28.690001,17.838600,...,NaN,17.809999,18.030001,8302.0,1.0,1.0,17.838600,NaN,2019-12-31,NaN
28021,58953M10,19140.0,56950.0,80786.0,3.0,9999.0,2019-12-31,9.700000,9.750000,9.740000,...,NaN,9.730000,9.750000,16372.0,1.0,1.0,9.740000,NaN,2019-12-31,NaN
28022,61023L10,19142.0,56978.0,79296.0,3.0,9999.0,2019-12-31,16.600000,27.000000,16.600000,...,NaN,16.570000,16.840000,10421.0,1.0,1.0,16.600000,NaN,2019-12-31,NaN
28023,65136T20,19143.0,56979.0,79336.0,3.0,9999.0,2019-12-31,-3.765000,-4.150000,-3.765000,...,NaN,3.580000,3.950000,11742.0,1.0,1.0,-3.765000,0.37,2019-12-31,NaN


In [7]:
# another example: get 500 stocks with largest market cap at a given month
from pandas.tseries.offsets import BDay
import pandas as pd

start_date = "2015-01-01"
end_date = "2019-12-31"
for date in pd.date_range(start_date, end_date, freq='BM'):
    print(date)
    df = db.raw_sql(f"SELECT permno, date, prc, shrout FROM crsp.msf "
                    f"where date='{date.strftime('%Y-%m-%d')}' "
                    f"AND prc IS NOT NULL "
                    f"ORDER BY abs(prc)*shrout DESC "
                    f"LIMIT 500")
    # sometimes, it's easier to fetch the raw DataFrame first, then clean/process it
    break

2015-01-30 00:00:00


In [8]:
# check the result
df['market cap'] = df['prc'].abs() * df['shrout']
df

,permno,date,prc,shrout,market cap
0,14593.0,2015-01-30,117.160004,5824748.0,6.824275e+08
1,11850.0,2015-01-30,87.419998,4234529.0,3.701825e+08
2,10107.0,2015-01-30,40.400002,8203785.0,3.314329e+08
3,22111.0,2015-01-30,100.139999,2799110.0,2.803029e+08
4,55976.0,2015-01-30,84.980003,3223190.0,2.739067e+08
...,...,...,...,...,...
495,85035.0,2015-01-30,73.870003,148469.0,1.096741e+07
496,76082.0,2015-01-30,26.500000,413020.0,1.094503e+07
497,75489.0,2015-01-30,17.049999,640320.0,1.091746e+07
498,89408.0,2015-01-30,58.930000,184037.0,1.084530e+07


In [9]:
# check what the largest stock is
db.raw_sql("SELECT * FROM crsp.dsfhdr WHERE permno=14593")

,permno,permco,hshrcd,dlstcd,hcusip,htick,hcomnam,htsymbol,hnaics,hprimexc,...,begvol,endvol,begbid,endbid,begask,endask,begopr,endopr,hsicmg,hsicig
0,14593.0,7.0,11.0,100.0,03783310,AAPL,APPLE INC,AAPL,334220,Q,...,1982-11-01,2019-12-31,1982-11-01,2019-12-31,1982-11-01,2019-12-31,1992-06-15,2019-12-31,35.0,357.0


In [14]:
# or to map each permno to a ticker (many ways to do this)
symbol = db.raw_sql('SELECT permno, htsymbol FROM crsp.dsfhdr;')
df['ticker'] = df['permno'].map(symbol.set_index('permno')['htsymbol'].to_dict())
df

,permno,date,prc,shrout,market cap,ticker
0,14593.0,2015-01-30,117.160004,5824748.0,6.824275e+08,AAPL
1,11850.0,2015-01-30,87.419998,4234529.0,3.701825e+08,XOM
2,10107.0,2015-01-30,40.400002,8203785.0,3.314329e+08,MSFT
3,22111.0,2015-01-30,100.139999,2799110.0,2.803029e+08,JNJ
4,55976.0,2015-01-30,84.980003,3223190.0,2.739067e+08,WMT
...,...,...,...,...,...,...
495,85035.0,2015-01-30,73.870003,148469.0,1.096741e+07,QRVO
496,76082.0,2015-01-30,26.500000,413020.0,1.094503e+07,COG
497,75489.0,2015-01-30,17.049999,640320.0,1.091746e+07,SPLS
498,89408.0,2015-01-30,58.930000,184037.0,1.084530e+07,MWE


### Script to download additional price data from yahoo finance:

In [14]:
import requests
import numpy as np
import pandas as pd
from datetime import datetime

# yahoo finance identifies securities by ticker, which is availabe in crsp.dsfhdr
def get_price_history_yahoo_finance(ticker, frequency='1d', begin_time=1546322400, finish_time=int(datetime.now().timestamp())):
    """
        read price data from yahoo finance
        args:
            ticker
            frequency: '1d' or '1wk' or '1mo'
            begin_time: int, epoch time, default is 1/1/2019
            finish_time: int, epoch time, default is current time
    """
    # define endpoint for making web request
    endpoint = fr"https://query1.finance.yahoo.com/v7/finance/download/{ticker}"
    param_dict = {
        'period1': begin_time,
        'period2': finish_time,
        'interval': frequency,
        'events': 'history',
        'includeAdjustedClose': 'true'
    }
    response = requests.get(url=endpoint, params=param_dict)
    if response.status_code == 200:
        df = pd.read_csv(response.url)
    else:
        raise Exception(f"broken url for {ticker}")

    df = df[['Date', 'Adj Close', 'Volume']].astype({'Date': np.datetime64}).set_index('Date')
    return df

In [15]:
get_price_history_yahoo_finance("AAPL").tail()

,Adj Close,Volume
Date,,
2020-11-20,117.339996,73391400
2020-11-23,113.849998,127959300
2020-11-24,115.169998,113874200
2020-11-25,116.029999,76499200
2020-11-27,116.589996,46691331


# 4. Get fundamental data

- comp.funda [overview](https://wrds-www.wharton.upenn.edu/pages/support/manuals-and-overviews/compustat/north-america-global-bank/wrds-overview-compustat-north-america-global-bank/#database-notes)
- ibes.statsum_epsus (analyst forecasts and earnings annoucement) [overview](https://wrds-www.wharton.upenn.edu/pages/support/manuals-and-overviews/i-b-e-s/ibes-estimates/general/wrds-overview-ibes/)
- etc.

Overview on Compustat: [Link](https://wrds-www.wharton.upenn.edu/pages/support/manuals-and-overviews/compustat/)
1. uniquely identified by `gvkey` on the company level and `iid` on the security level. One `gvkey` may correspond to multiple `iid`. In terms of fundamental data, `gvkey` often suffices as different classes of stocks are tied to the same company

Most commonly used datasets in i/b/e/s:
![pic](images/common_ibes.png)

**Note**: i/b/e/s's data on earnings call date are not complete. Trading volumes would be a better indicator of past earnings annoucement.

In [16]:
# lots of features
db.describe_table('comp', 'funda')

Approximately 822620 rows in comp.funda.


,name,nullable,type
0,gvkey,True,VARCHAR(24)
1,datadate,True,DATE
2,fyear,True,DOUBLE PRECISION
3,indfmt,True,VARCHAR(48)
4,consol,True,VARCHAR(8)
...,...,...,...
943,au,True,VARCHAR(32)
944,auop,True,VARCHAR(32)
945,auopic,True,VARCHAR(4)
946,ceoso,True,VARCHAR(4)


In [26]:
# an example: get historical earnings call date
db.raw_sql(f"SELECT DISTINCT anndats_act FROM ibes.statsum_epsus "
           f"WHERE cusip='03783310' AND anndats_act IS NOT NULL ORDER BY anndats_act")

,anndats_act
0,1983-10-25
1,1984-10-19
2,1985-01-18
3,1985-04-15
4,1985-07-18
...,...
139,2019-07-30
140,2019-10-30
141,2020-01-28
142,2020-04-30


# 5. Link CRSP, Compustat and others
Identifiers for different vendors/databases:
1. CRSP: permco/permno
2. Compustat: gvkey/iid
3. SEC: cik
4. S&P Global Market Intelligence: CUSIP

#### [Link CRSP and Compustat ](https://wrds-www.wharton.upenn.edu/pages/support/applications/linking-databases/linking-crsp-and-compustat/)
#### [Link IBES and CRSP](https://wrds-www.wharton.upenn.edu/pages/support/applications/linking-databases/linking-ibes-and-crsp-data/)

In [28]:
# linking CRSP and Compustat essentially uses the table that contains each databses's identifiers
# crsp.ccmxpf_linktable
db.describe_table('crsp', 'ccmxpf_linktable')

Approximately 77688 rows in crsp.ccmxpf_linktable.


,name,nullable,type
0,gvkey,True,VARCHAR(6)
1,linkprim,True,VARCHAR(1)
2,liid,True,VARCHAR(3)
3,linktype,True,VARCHAR(2)
4,lpermno,True,DOUBLE PRECISION
5,lpermco,True,DOUBLE PRECISION
6,usedflag,True,DOUBLE PRECISION
7,linkdt,True,DATE
8,linkenddt,True,DATE


In [32]:
# there's no table that contains both permno and IBES Ticker, but one can connect use either
# exchange ticker, company name or CUSIP, depending on the specific table of crsp you're using
db.raw_sql("SELECT * FROM crsp.msf as a, ibes.statsum_epsus as b WHERE a.cusip=b.cusip LIMIT 10")

,cusip,permno,permco,issuno,hexcd,hsiccd,date,bidlo,askhi,prc,...,highest,lowest,usfirm,fpedats,actual,actdats_act,acttims_act,anndats_act,anntims_act,curr_act
0,87482X10,14471.0,54747.0,73177.0,3.0,9999.0,2014-01-31,NaN,NaN,NaN,...,0.1,0.07,1.0,2014-03-31,0.12,2014-05-12,40087.0,2014-05-06,38700.0,USD
1,87482X10,14471.0,54747.0,73177.0,3.0,9999.0,2014-02-28,13.35,14.00,13.88,...,0.1,0.07,1.0,2014-03-31,0.12,2014-05-12,40087.0,2014-05-06,38700.0,USD
2,87482X10,14471.0,54747.0,73177.0,3.0,9999.0,2014-03-31,13.59,14.64,14.64,...,0.1,0.07,1.0,2014-03-31,0.12,2014-05-12,40087.0,2014-05-06,38700.0,USD
3,87482X10,14471.0,54747.0,73177.0,3.0,9999.0,2014-04-30,13.27,14.63,13.41,...,0.1,0.07,1.0,2014-03-31,0.12,2014-05-12,40087.0,2014-05-06,38700.0,USD
4,87482X10,14471.0,54747.0,73177.0,3.0,9999.0,2014-05-30,13.21,13.93,13.57,...,0.1,0.07,1.0,2014-03-31,0.12,2014-05-12,40087.0,2014-05-06,38700.0,USD
5,87482X10,14471.0,54747.0,73177.0,3.0,9999.0,2014-06-30,13.62,14.44,13.79,...,0.1,0.07,1.0,2014-03-31,0.12,2014-05-12,40087.0,2014-05-06,38700.0,USD
6,87482X10,14471.0,54747.0,73177.0,3.0,9999.0,2014-07-31,13.12,14.15,13.39,...,0.1,0.07,1.0,2014-03-31,0.12,2014-05-12,40087.0,2014-05-06,38700.0,USD
7,87482X10,14471.0,54747.0,73177.0,3.0,9999.0,2014-08-29,13.20,14.66,14.66,...,0.1,0.07,1.0,2014-03-31,0.12,2014-05-12,40087.0,2014-05-06,38700.0,USD
8,87482X10,14471.0,54747.0,73177.0,3.0,9999.0,2014-09-30,13.83,14.91,13.83,...,0.1,0.07,1.0,2014-03-31,0.12,2014-05-12,40087.0,2014-05-06,38700.0,USD
9,87482X10,14471.0,54747.0,73177.0,3.0,9999.0,2014-10-31,13.50,14.04,13.98,...,0.1,0.07,1.0,2014-03-31,0.12,2014-05-12,40087.0,2014-05-06,38700.0,USD


# 6. Find desired database